# Asset Management: AI-Powered Research Pipeline Demo

This demo showcases how we've transformed our research workflows using Snowflake Cortex AI to deliver faster insights and better investment decisions.

## Executive Summary

Deploy a comprehensive AI research platform that processes, classifies, and intelligently searches across our entire research library. This solution has revolutionized how our analysts discover insights and make investment decisions.

## Key Steps in the Pipeline

**1. Automated Document Processing** - All research documents (earnings calls, SEC filings, analyst reports) are ingested and processed

**2. AI-Powered Classification** - Documents are instantly categorized by sector, document type, and investment themes

**3. Intelligent Knowledge Base** - Structured storage with metadata for advanced filtering and analysis

**4. Semantic Search Engine** - Natural language queries across the entire research library

**5. Interactive Research Assistant** - Conversational AI interface for analysts to get instant answers

## Solution Architecture
Our platform includes:
- **Automated Document Ingestion**: Processes PDFs, earnings reports, research notes, and regulatory filings in real-time
- **Industry Classification System**: AI automatically categorizes documents by GICS sectors and investment themes
- **Intelligent Search Interface**: RAG-powered conversational assistant for querying investment documents
- **Compliance & Security**: Built-in data governance with audit trails and access controls
- **Scalable Infrastructure**: Handles thousands of documents with auto-scaling compute resources

## Why Snowflake Makes This Easier and Faster

**🚀 Integrated AI Stack**: Unlike traditional solutions requiring multiple vendors and complex integrations, Snowflake Cortex provides LLMs, embeddings, and vector search natively within our data warehouse. This eliminates data movement, reduces complexity, and accelerates time-to-value.

**⚡ No Infrastructure Management**: Traditional document AI requires managing GPU clusters, vector databases, and ML pipelines. Snowflake handles all infrastructure automatically, letting our team focus on investment decisions rather than IT operations.

**💰 Elastic Economics**: Instead of paying for idle GPU infrastructure, we only pay for compute when processing documents. During busy earnings seasons, the platform auto-scales; during quiet periods, costs drop to near zero.

**🔒 Enterprise Security**: Financial services require the highest security standards. Snowflake provides SOC 2 Type II compliance, encryption at rest and in transit, and role-based access controls out of the box - no additional security infrastructure needed.

**📊 Data Gravity**: Our research documents stay in the same platform as our portfolio data, market data, and analytics. This enables cross-functional analysis that would be impossible with separate systems.

In [ ]:
-- Set Context
USE DATABASE ASSET_MANAGEMENT_AI;
USE SCHEMA RESEARCH_ANALYTICS;
USE WAREHOUSE ASSET_MANAGEMENT_AI_WH;
ALTER SESSION SET query_tag = '{"origin":"sf_sit-is","name":"asset_management_ai:_intelligent_research_platform","version":{"major":1,"minor":0},"attributes":{"is_quickstart":1,"source":"python"}}';

**Document Upload:**

As part of setup, you should have uploaded relevant documents.

**Next Steps: Automated Upload via Data Pipeline**
For production environments, integrate with your existing research systems using Snowflake connectors.

**Security Note**: All documents are encrypted at rest and in transit. Access is controlled via Snowflake's role-based security model, ensuring only authorized analysts can access sensitive research.

Let's verify your documents are uploaded:

In [ ]:
-- Document Verification Queries
-- List all research documents in the staging area
-- This provides visibility into what documents are available for processing
--ls @research_docs_stage;

-- Get detailed information about uploaded documents
SELECT file_url, 
       SIZE, 
       last_modified,
       REGEXP_SUBSTR(file_url, '[^/]+$') as document_name
FROM directory(@research_docs)
ORDER BY last_modified DESC;

# Document Pre-processing for Vector Search

Preparing a document for vector search involves two main steps: parsing and chunking.

## Step 1: Parse the Document

First, use the `CORTEX.PARSE_DOCUMENT` function to convert raw pdfs into a structured format. This function extracts the document's content and metadata, making it ready for the next stage.

For more details on the function, see the [CORTEX.PARSE_DOCUMENT documentation](https://www.google.com/search?q=https://www.example.com/cortex-docs).

## Step 2: Chunk the Document

Next, break the parsed content into smaller segments or "chunks." using the `CORTEX.SPLIT_TEXT_RECURSIVE_CHARACTERPARSE_DOCUMENT` function This is essential because:

* **Improved Search Relevance**: Smaller chunks lead to more precise search results.

* **Embedding Efficiency**: It ensures each segment fits the size limits of vector embedding models.

* **Contextual Integrity**: Chunks are sized to maintain context while being easy to process.

Chunk sizes can be defined by character count, word count, or sentence count, depending on your needs.


In [ ]:
CREATE OR REPLACE TABLE research_knowledge_base_raw_documents ( 
    relative_path VARCHAR(16777216),        -- Document file path
    document_name VARCHAR(500),              -- Clean document name
    file_size NUMBER(38,0),                  -- File size for audit purposes
    file_url VARCHAR(16777216),              -- Direct file access URL
    scoped_file_url VARCHAR(16777216),       -- Security-scoped URL
    file_content VARCHAR(16777216)          -- full text from document
  );


--Parse documents into a single cell within a table
INSERT INTO research_knowledge_base_raw_documents (
       relative_path, document_name, file_size, file_url, scoped_file_url, file_content) 
SELECT
   d.relative_path,
   REGEXP_SUBSTR(d.relative_path, '[^/]+$') as document_name,
   d.size as file_size,
   d.file_url, 
   BUILD_SCOPED_FILE_URL(@research_docs, d.relative_path) as scoped_file_url,
   TO_VARCHAR(SNOWFLAKE.CORTEX.PARSE_DOCUMENT(
                    @research_docs, 
                    relative_path, 
                    {'mode': 'LAYOUT'})) as file_content
FROM
   directory(@research_docs) d;

In [ ]:
-- Knowledge Base Table Creation and Population
-- Create the comprehensive investment research table
CREATE OR REPLACE TABLE research_knowledge_base ( 
    relative_path VARCHAR(16777216),        -- Document file path
    document_name VARCHAR(500),              -- Clean document name
    file_size NUMBER(38,0),                  -- File size for audit purposes
    file_url VARCHAR(16777216),              -- Direct file access URL
    scoped_file_url VARCHAR(16777216),       -- Security-scoped URL
    chunk VARCHAR(16777216),                 -- Searchable text content
    chunk_index NUMBER(10,0),                -- Chunk sequence for document reconstruction
    
    -- Investment-specific classifications
    industry_sector VARCHAR(100),            -- GICS sector classification
    document_type VARCHAR(100),              -- Earnings, Research, Filing, etc.
    company_ticker VARCHAR(10),              -- Stock ticker if applicable
    coverage_universe VARCHAR(50),           -- Large Cap, Mid Cap, Small Cap
    
    -- Temporal and source tracking
    document_year INTEGER,                      -- year document was created/published
    upload_timestamp TIMESTAMP_NTZ DEFAULT CURRENT_TIMESTAMP(),
    analyst_name VARCHAR(100),               -- Source analyst or firm
    
    -- Compliance and governance
    confidentiality_level VARCHAR(20) DEFAULT 'INTERNAL',  -- INTERNAL, CONFIDENTIAL, RESTRICTED
    retention_date DATE,                     -- For compliance retention policies
    access_tags ARRAY                        -- For fine-grained access control
);

-- Insert processed chunks with document intelligence
INSERT INTO research_knowledge_base 
   (relative_path, document_name, file_size, file_url, scoped_file_url, chunk)
 SELECT
   relative_path,
   document_name,
   file_size,
   file_url, 
   scoped_file_url,
   c.value as chunk
FROM
   research_knowledge_base_raw_documents,
   LATERAL FLATTEN( input => SNOWFLAKE.CORTEX.SPLIT_TEXT_RECURSIVE_CHARACTER (file_content,
      'none',
      2000,
      400
   )) c;

  
--Verify the processing results
SELECT COUNT(*) as total_chunks,
       COUNT(DISTINCT relative_path) as total_documents,
       AVG(LENGTH(chunk)) as avg_chunk_length
FROM Research_analytics.research_knowledge_base;

# AI-Powered Industry Classification for Investment Documents

**Why This Revolutionizes Investment Research Organization:**

Manual document classification is the biggest bottleneck in research workflows. Analysts spend 30-40% of their time just finding and organizing information. Our AI-powered classification system:

**Transformational Benefits:**
- **Zero Manual Effort**: Instant classification of new documents as they arrive
- **Consistent Standards**: Eliminates subjective classification differences between analysts
- **Multi-Dimensional Tagging**: Documents classified by industry, document type, and investment theme
- **Dynamic Updates**: Classifications can be refined as market conditions evolve

**Investment-Specific Classification Model:**
We'll classify documents across multiple dimensions relevant to investment decision-making:
- **GICS Sectors**: Technology, Healthcare, Financials, Energy, etc.
- **Document Types**: Earnings transcripts, research reports, regulatory filings
- **Investment Themes**: Growth, Value, ESG, Dividend, etc.
- **Market Segments**: Large Cap, Mid Cap, Small Cap, International

**The AI Advantage:**
Using Snowflake's Cortex LLMs, we can process thousands of documents in minutes, not weeks.


In [ ]:
-- AI-Powered Document Classification
-- Create intelligent industry classification using advanced LLM reasoning
CREATE OR REPLACE TEMPORARY TABLE document_classifications 
AS WITH unique_documents AS (
  SELECT DISTINCT 
    relative_path,
    document_name,
    -- Extract potential company ticker from filename
    REGEXP_SUBSTR(UPPER(document_name), '[A-Z]{2,5}') as potential_ticker,
    -- Extract date patterns for temporal analysis
    REGEXP_SUBSTR(document_name, '\\d{4}') as potential_year
  FROM Asset_management_ai.Research_analytics.research_knowledge_base
),
document_intelligence AS (
  SELECT
    relative_path,
    document_name,
    potential_ticker,
    potential_year,
    
    -- Multi-faceted AI classification using advanced prompting
    TRIM(SNOWFLAKE.CORTEX.COMPLETE(
      'llama3.1-70b',
      'You are an expert investment analyst. Analyze this document filename and classify it into ONE of these GICS sectors: ' ||
      'Technology, Healthcare, Financials, Energy, ESG, Emerging_Markets, Consumer_Discretionary, Consumer_Staples, ' ||
      'Utilities, Real_Estate, Communication_Services. ' ||
      'Consider company names, industry keywords, and business context. ' ||
      'Filename: ' || document_name || 
      ' Return ONLY the sector name, no explanation.'
    ), '\n') AS industry_sector,
    
    -- Document type classification
    TRIM(SNOWFLAKE.CORTEX.COMPLETE(
      'claude-3-5-sonnet',
      'Classify this investment document type into ONE category: ' ||
      'Earnings_Call, Research_Report, SEC_Filing, Company_Report, Industry_Analysis, News_Article, Other. ' ||
      'Filename: ' || document_name ||
      ' Return ONLY the document type, no explanation.'
    ), '\n') AS document_type,
    
    -- Investment theme detection
    TRIM(SNOWFLAKE.CORTEX.COMPLETE(
      'mistral-large2',
      'Identify the primary investment theme from this document name. Choose ONE: ' ||
      'Growth, Value, Dividend, ESG, Small_Cap, Large_Cap, International, Sector_Rotation, Market_Analysis, Company_Specific. ' ||
      'Filename: ' || document_name ||
      ' Return ONLY the theme, no explanation.'
    ), '\n') AS investment_theme
    
  FROM unique_documents
)
SELECT * FROM document_intelligence;

-- Review the AI classification results
select * from document_classifications;

# Enrich the Knowledge Base with Investment Intelligence

**Why This Step Creates Competitive Advantage:**

Raw documents are just data. Classified, enriched, and connected documents become actionable intelligence. This step transforms your research library into a strategic asset:

**Intelligence Amplification:**
- **Sector Rotation Insights**: Quickly identify which sectors are getting the most research attention
- **Temporal Analysis**: Track how sentiment and coverage evolve over time
- **Coverage Gaps**: Identify underresearched opportunities in your universe
- **Compliance Mapping**: Ensure research coverage meets regulatory requirements

**Multi-Dimensional Search Capabilities:**
After enrichment, analysts can ask questions like:
- "Show me all ESG-related research on Technology companies from Q4 2024"
- "What are the key risks mentioned in Energy sector earnings calls this quarter?"
- "Compare dividend sustainability analysis across Utilities holdings"

Let's enrich the knowledge base with this investment intelligence:


In [ ]:
-- Knowledge Base Enrichment
-- Enrich the knowledge base with comprehensive investment classifications
UPDATE research_knowledge_base 
SET 
    industry_sector = dc.industry_sector,
    document_type = dc.document_type,
    -- Extract and standardize company ticker
    company_ticker = CASE 
        WHEN dc.potential_ticker IS NOT NULL AND LENGTH(dc.potential_ticker) BETWEEN 2 AND 5 
        THEN dc.potential_ticker
        ELSE NULL 
    END,
    -- Parse and validate document date
    document_year = dc.potential_year,
    -- Classify coverage universe based on common conventions
    -- Set default analyst attribution
    analyst_name = CURRENT_USER()
FROM document_classifications dc
WHERE research_knowledge_base.relative_path = dc.relative_path;

-- Create summary statistics for portfolio management
SELECT 
    'Document Distribution by Sector' as metric_category,
    industry_sector,
    COUNT(DISTINCT relative_path) as unique_documents,
    COUNT(*) as total_chunks,
    AVG(LENGTH(chunk)) as avg_chunk_size
FROM research_knowledge_base
WHERE industry_sector IS NOT NULL
GROUP BY industry_sector
ORDER BY unique_documents DESC;

# Intelligent Search Engine Deployment

**The Search Revolution:**

Analysts were drowning in keyword search results. Searching for "inflation hedging" missed documents discussing "purchasing power protection" or "real asset allocation." Critical insights were buried in irrelevant results, and urgent research questions took hours to answer during volatile market conditions.

**Semantic Search Breakthrough:**

Snowflake Cortex Search understands investment concepts and relationships. When analysts ask about "dividend sustainability," it finds discussions about "payout ratios," "free cash flow coverage," and "capital allocation priorities" - even when those exact terms weren't used in the query.

**Revolutionary Research Capabilities:**
- **Conceptual Intelligence**: Understands investment terminology and finds conceptually related content
- **Cross-Document Discovery**: Automatically surfaces insights from earnings calls, research reports, and SEC filings simultaneously  
- **Context-Aware Prioritization**: Ranks results by relevance to specific investment questions
- **Real-Time Research**: New documents are instantly searchable without manual indexing

**Infrastructure Advantages:**
- **Zero DevOps**: Fully managed service with automatic scaling during earnings season
- **Integrated Security**: Inherits GAM's existing Snowflake access controls and compliance framework
- **Cost Efficiency**: Pay only for actual searches, not idle search infrastructure
- **Enterprise Governance**: Native integration with data lineage and audit requirements

In [ ]:
-- Cortex Search Service Deployment
-- Deploy the investment research search service
CREATE OR REPLACE CORTEX SEARCH SERVICE Asset_management_ai.Research_analytics.investment_search_svc
ON chunk                                    -- Primary search field containing document content
ATTRIBUTES 
    industry_sector,                        -- Filter by investment sector
    document_type,                          -- Filter by document type (earnings, research, etc.)
    company_ticker,                         -- Filter by specific companies
    coverage_universe,                      -- Filter by market cap segment
    document_year,                          -- Filter by time period
    analyst_name                            -- Filter by research source
WAREHOUSE = asset_management_ai_wh          -- Dedicated compute for search operations
TARGET_LAG = '60 minutes'                   -- Near real-time updates for new documents
AS
    SELECT 
        chunk,                              -- Searchable content
        relative_path,                      -- Document identification
        document_name,                      -- Human-readable document name
        file_url,                           -- Direct document access
        industry_sector,                    -- Investment classification
        document_type,                      -- Document category
        company_ticker,                     -- Company identifier
        coverage_universe,                  -- Market segment
        document_year,                      -- Temporal classification
        analyst_name,                       -- Source attribution
        chunk_index                         -- For document reconstruction
    FROM research_knowledge_base;

-- Verify search service deployment
DESC CORTEX SEARCH SERVICE investment_search_svc;

# GAM Analysts Using the Research Assistant

**Real Analyst Queries from GAM's Daily Operations:**

Experience how GAM's investment team uses our AI-powered research platform for live investment decisions:

Use Snowflake Intelligence to query the document research library in the context of your current portfolio

**📊 Portfolio Analysis Queries:**
- "What are the primary growth catalysts mentioned for our Technology holdings?"
- "Which Healthcare companies in our portfolio and what innovation opportunities exist?"

**🏢 Investment Decision Support:**
- "Summarize our investment portfolio and outline the risk associated with each" 
- "Chart our portfolio holdings by sector and risk?"

**💡 Strategic Portfolio Insights:**
- "What ESG trands are likely to impact our portfolio?"
- "Provide a 2 paragraph summary of our Technology Sector Overview report and 5 points where this will impact our portfolio?"

**GAM's Research Platform Results:**

** ⚡ Decision Speed**: Investment insights in 30 seconds vs. hours of manual research

** 🎯 Research Precision**: AI understands investment terminology and finds conceptually related insights

** 📚 Comprehensive Coverage**: Simultaneous search across earnings calls, SEC filings, and analyst reports  

** 🔄 Contextual Intelligence**: Build complex investment theses through conversational follow-up questions

** 📋 Automated Organization**: Results filtered by sector, document type, and time period automatically


## Professional Asset Management Workflow Portfolio and Scenario Analysis

Next this demonstrates a comprehensive **asset management analytics workflow** that combines data protection, AI-driven insights, and portfolio intelligence while maintaining enterprise-grade data integrity and operational safety standards.

---

## 🔑 Why Zero-Copy Cloning is Critical for Asset Management

### The Challenge Every Analyst Faces
In asset management, **data integrity is non-negotiable**. Analysts need to:
- Test "what-if" portfolio scenarios without affecting live positions
- Model allocation changes for client presentations  
- Perform backtesting and stress testing
- Collaborate with team members on shared analysis
- Experiment with new analytical approaches

### The Traditional Problem
Before Snowflake CLONE, analysts faced impossible choices:
- ❌ Work directly on production data (too risky)
- ❌ Create full data copies (expensive and time-consuming)
- ❌ Export to external tools (security concerns and data latency)
- ❌ Wait for IT to provision sandbox environments (too slow)

### The Snowflake CLONE Solution
**Zero-copy cloning** solves all these problems instantly:
- ⚡ **Instant creation**: Sandbox ready in seconds, not hours
- 💰 **Zero initial cost**: No data duplication means no storage cost until you modify data
- 🛡️ **Complete safety**: Production data remains untouched
- 🧪 **Unlimited experimentation**: Test any scenario with confidence
- 👥 **Team collaboration**: Every analyst can have their own sandbox
- 🔄 **Easy reset**: Recreate clean environment anytime

In [ ]:
-- Create a zero-copy clone of our existing portfolio holdings for analysis and manipulation
-- Using Snowflake CLONE: instant creation, no data duplication, no additional storage cost initially
CREATE OR REPLACE TABLE PORTFOLIO_HOLDINGS_ANALYSIS 
CLONE PORTFOLIO_HOLDINGS;

-- Verify the clone was created successfully (notice the instant execution!)
SELECT COUNT(*) as TOTAL_HOLDINGS, 
       COUNT(DISTINCT PORTFOLIO_NAME) as UNIQUE_PORTFOLIOS,
       SUM(MARKET_VALUE) as TOTAL_AUM
FROM PORTFOLIO_HOLDINGS_ANALYSIS;

## 📊 Portfolio Modification and Analysis

1. **Adding a New Position**: Insert a hypothetical technology investment (OpenAI Corp) to the Growth Fund Alpha portfolio to test how it affects sector concentration and risk profile

2. **Adjusting Existing Holdings**: Increase the Microsoft Corporation position weight by 10% to simulate a rebalancing scenario and assess the impact on portfolio allocation


In [ ]:
-- Now we can safely modify our cloned data - this is when storage costs begin
-- Example: Add a hypothetical new position for testing AI_AGG capabilities
INSERT INTO PORTFOLIO_HOLDINGS_ANALYSIS VALUES
('H999', 'Growth Fund Alpha', 'OpenAI Corp', 'Technology', 500000.00, 1.67, 'High', CURRENT_DATE());

-- Example: Update a position weight for scenario analysis  
-- Only the modified rows will consume additional storage
UPDATE PORTFOLIO_HOLDINGS_ANALYSIS 
SET WEIGHT_PERCENT = WEIGHT_PERCENT * 1.10,  -- Simulate 10% increase
    MARKET_VALUE = MARKET_VALUE * 1.10
WHERE SECURITY_NAME = 'Microsoft Corporation';

-- Verify our working copy has been updated (production data remains unchanged)
SELECT 'Modified Holdings' as STATUS, COUNT(*) as TOTAL_RECORDS 
FROM PORTFOLIO_HOLDINGS_ANALYSIS;

---

# 📊 AI_AGG: Portfolio Analytics and Aggregation

## Use Case for Asset Management
**AI_AGG** is a game-changer for Asset Management because it transforms complex SQL queries into natural language conversations with your data. It's primary purpose is to apply an aggregated function across a group of rows to make predictions or classify data, without the need to export data or use external ML platforms.

### Common Asset Management Questions AI_AGG Can Answer:
- "What percentage of my portfolio is in high-risk securities?"
- "How diversified is this portfolio across sectors and geographies?"  
- "Which holdings are contributing most to portfolio volatility?"
- "What's the ESG score distribution across my holdings?"

In [ ]:
WITH PortfolioAggregatedDetails AS (
    SELECT
        PORTFOLIO_NAME,
        LISTAGG(
            'Security: ' || SECURITY_NAME ||
            ', Weight: ' || WEIGHT_PERCENT || '%' ||
            ', Risk: ' || RISK_LEVEL ||
            ', Sector: ' || SECTOR,
            '; '
        ) WITHIN GROUP (ORDER BY SECURITY_NAME) AS AGGREGATED_SECURITY_DETAILS
    FROM PORTFOLIO_HOLDINGS_ANALYSIS
    GROUP BY PORTFOLIO_NAME
)
SELECT
    PAD.PORTFOLIO_NAME,
    AI_AGG(
        PAD.AGGREGATED_SECURITY_DETAILS, -- Now this is a single string per portfolio from the CTE
        'What is the overall risk profile of this portfolio based on individual security risk levels and concentrations? Provide a risk score from 1-10. Output only the score and rationale, nothing else.'
    ) AS PORTFOLIO_RISK_ASSESSMENT
FROM PortfolioAggregatedDetails PAD
GROUP BY PAD.PORTFOLIO_NAME
ORDER BY PAD.PORTFOLIO_NAME;

---

# 🎯 Putting it all together in an Interactive Asset Management Dashboard with Streamlit

The interactive Streamlit application provides real-time portfolio analysis, AI insights, and dynamic visualizations for asset management professionals.

## 🔍 What Does This Dashboard Show You?

This dashboard is your AI-powered command center for investment management, answering critical questions:

**"Which portfolios are performing well?"** → Performance metrics at the top

**"What does AI think about our investments?"** → Color-coded sentiment analysis (🟢🟡🔴)

**"Where are our biggest risks?"** → Risk scores and real-time alerts

**"What do the latest research reports recommend?"** → AI document analysis

**"How should we adjust our strategy?"** → Dynamic AI recommendations